# C. Coelho, M. F. P. Costa, and L.L. Ferrás, "The role of adaptive activation functions in Fractional Physics-Informed Neural Networks" 

In [18]:
%%capture
!git clone https://github.com/CeciliaCoelho/fPINNs.git
!python setup.py install

In [19]:
%%capture
pip install scikit-optimize matplotlib pandas

In this work we aim to analyse the effect of adaptive activation functions on the convergence rate and solution
accuracy of fPINNs. For this purpose, we consider the numerical solution of the fractional Laplacian equation (FLE),

\begin{equation}
    (-\Delta)^{\alpha/2}u(x)=f(x),~~~u(0)=u(1)=0,~~~x\in(0,1),~\alpha\in(1,2) \tag{1}
\end{equation}
where $(-\Delta)^{\alpha/2}$ is the fractional Laplacian operator (see <a href="http://example.com/" target="_blank">link</a> for more details). <span style="color:red">add link to go to icnaam 2021 paper</span>

we analyse the performance of fPINNs when one adaptive parameter per layer is used in solving the
1D Fractional Laplacian Equation with a non-smooth analytical solution. Moreover, we analyse the limitations of
fPINNs when solving the FLE with a series of gradually smaller non-integer order α values.

In [20]:
%%capture
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from scipy.special import gamma
import matplotlib.pyplot as plt
import pandas as pd

import deepxde as dde
from deepxde.backend import tf

We consider the numerical solution of (1) with $f(x)=\left(2\cos\left(\frac{\pi\alpha}{2}\right)\right)\Gamma (\alpha + 2)x$. The analytical solution is not smooth, and is given by $u(x)=x(1-x^2)^{\alpha /2}$. As we have seen in an earlier work \cite{ICNAAM2021}, this case is more extreme and should present greater difficulties in approximating the fractional operator (through the difference scheme) and in solving the equation by fPINN.


In [10]:
def NN(activation, optimizer, initializer, alpha):
    
    def fle(x, y, int_mat):
        # defining the fractional Laplacian equation(1)
        if isinstance(int_mat, (list, tuple)) and len(int_mat) == 3:
            int_mat = tf.SparseTensor(*int_mat)
            lhs = tf.sparse_tensor_dense_matmul(int_mat, y)
        else:
          
          lhs = tf.matmul(int_mat, y)
         
        lhs /= 2 * np.cos(alpha * np.pi / 2)
        rhs = gamma(alpha + 2) * x
        return lhs - rhs[: tf.size(lhs)]

    def NNSol(x):
        # defining the forced non-smooth numerical solution
        return x * (np.abs(1 - x**2)) ** (alpha / 2)
        

    geom = dde.geometry.Interval(0, 1)
    bc = dde.DirichletBC(geom, NNSol, lambda _, on_boundary: on_boundary)


    data = dde.data.FPDE(geom, fle, alpha, bc, [101], meshtype="static", solution=NNSol, num_domain=40)
    
    # network architecture
    net = dde.maps.FNN([1] + [128] + [64] + [32] + [16] + [8] + [1], activation, initializer)
    net.apply_output_transform(lambda x, y: x * (1 - x) * y)

    # training
    model = dde.Model(data, net)
    model.compile(optimizer, metrics=["l2 relative error"])
    losshistory, train_state = model.train(epochs=50000)

    # testing
    X = geom.random_points(1000)
    y_true = NNSol(X)
    y_pred = model.predict(X)
    
    model.print_model()
    
    return losshistory 

We considered a feed-forward NN with 5 hidden layers and a configuration of 128-64-32-16-8 neurons.
In our work training waas done with different combinations of hyperparameters:
- activation function: tanh, silu;
- optimizer: L-BFGS;
- initializer (weights): Glorot Normal with mean and standard deviation given respectively by $0$ and  $\sqrt{\frac{2}{n^{k}+n^{k-1}}}$ ($n^j$ is the number of neurons in the layer $j=k-1,k$) and uniform distributions given by $U(-1,1)$ and $U(-2,2)$;
- alpha: 1.9,1.8,1.7,1.6,1.5,1.4,1.3,1.2;

we implemented a multi-start strategy to avoid being trapped in a local optimal solution

In [11]:
def multiStart(n, activation, optimizer, initializer, l_alpha):
    res = {}
    for j in range(len(l_alpha)):
        res[str(l_alpha[j])] = []
        for i in range(n):
            res[str(l_alpha[j])].append(NN(activation, optimizer, initializer, l_alpha[j]))
    return res

In [12]:
l_alpha = [1.9]#,1.8,1.7,1.6,1.5,1.4,1.3,1.2]

#### preprocess data

we can organize the L2 relative errors by alpha value in a dataframe by using simple preprocessing

In [13]:
def organize(results):
    for k, v in results.items():
        results[k] = list(map(lambda x: x.metrics_test[1][0], v))
    print(pd.DataFrame.from_dict(results))

## Glorot Normal

In [14]:
rGlorotTanh = multiStart(5, 'LAAF-1 tanh', 'L-BFGS',"Glorot normal", l_alpha)

Compiling model...
Building feed-forward neural network...
'build' took 0.069387 s



/tf/fPINNs/deepxde/nn/tensorflow_compat_v1/fnn.py:103: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 0.519487 s

Initializing variables...
!!!!Training model...

<built-in function vars>
<built-in function vars>
Step      Train loss              Test loss               Test metric   
0         [8.17e+00, 0.00e+00]    [8.17e+00, 0.00e+00]    [9.44e-01]    
INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Objective function value: 0.131728
  Number of iterations: 30
  Number of functions evaluations: 54
<built-in function vars>
<built-in function vars>
54        [1.32e-01, 0.00e+00]    [1.32e-01, 0.00e+00]    [7.56e-03]    

Best model at step 54:
  train loss: 1.32e-01
  test loss: 1.32e-01
  test metric: [7.56e-03]

'train' took 0.763950 s

final adaptive parameters values: [0.5958619, 0.58940727, 0.60133773, 1.0965881, 2.0483937]
Compiling model...
Building feed-forward neural network...
'build' took 0.051355 s

'compile' took 0.204468 s

Initializing variables...
!!!!Training model...

<built-in function vars>


KeyboardInterrupt: 

In [ ]:
organize(rGlorotTanh)

In [ ]:
rGlorotSilu = multiStart(5, 'LAAF-1 silu', 'L-BFGS',"Glorot normal",l_alpha)

In [ ]:
organize(rGlorotSilu)

## Random uniform [-1,1]

In [ ]:
initializer = tf.keras.initializers.RandomUniform(minval=-1, maxval=1, seed=None)

In [ ]:
rRandom1Tanh = multiStart(5, 'LAAF-1 tanh', 'L-BFGS',initializer,l_alpha)

In [ ]:
organize(rRandom1Tanh)

In [ ]:
rRandom1Silu = multiStart(5, 'LAAF-1 silu', 'L-BFGS',initializer,l_alpha)

In [ ]:
organize(rRandom1Silu)

## Random uniform [-2,2]

In [ ]:
initializer = tf.keras.initializers.RandomUniform(minval=-2, maxval=2, seed=None)

In [ ]:
rRandom2Tanh = multiStart(5, 'LAAF-1 tanh', 'L-BFGS',initializer,l_alpha)

In [ ]:
organize(rRandom2Tanh)

In [ ]:
rRandom2Silu = multiStart(5, 'LAAF-1 silu', 'L-BFGS',initializer,l_alpha)

In [ ]:
organize(rRandom2Silu)